In [1]:
import numpy as np
import networkx as nx
import json
import spektral
from spektral.data import Graph, Dataset
import matplotlib.pyplot as plt
import tensorflow as tf
import random
import pyproj

import utm
import numpy as np
import random
import math
import time

import pandas as pd
import pydeck as pdk

import pydeck as pdk
import pandas as pd
import ipywidgets
# use palettable for colormap colors
from palettable.cartocolors.sequential import BrwnYl_3
import json

Colors_RGB = [[31, 119, 180],
          [255, 127, 14],
          [44, 160, 44],
          [214, 39, 40],
          [148, 103, 189],
          [140, 86, 75],
          [227, 119, 194],
          [127, 127, 127],
          [188, 189, 34],
          [23, 190, 207]]
          
    
JFK_LAT_LON = 40.645944, -73.784839
Num_Vertiports = 28


In [2]:
g = nx.read_gml("vfrLatest.gml")

In [3]:
LongLat = nx.get_node_attributes(g, "pos")

In [4]:
g = nx.Graph()

In [5]:
lons = []
lats = []
Xs = []
Ys = []
New_Long_Lat = {}
for i in range(Num_Vertiports):
    lon = LongLat[str(i)][0]
    lat = LongLat[str(i)][1]
    
    x, y, z, zl = utm.from_latlon(lat, lon)
    
    lons.append(lon)
    lats.append(lat)

    Xs.append(x)
    Ys.append(y)
    g.add_node(i)
    
    New_Long_Lat[i] = [lon, lat]

In [6]:
x_c, y_c, z, zl = utm.from_latlon(lats[18], lons[18])

In [7]:
sep_dist = 804.672*2
size = 160934/4

In [8]:
y_s = y_c + size/2
x_s = x_c - size/2

In [9]:
Width = int(round(size/sep_dist))

In [10]:
grid_graph = nx.grid_graph((Width, Width))

In [11]:
node_num = Num_Vertiports
ind = {}
for n in grid_graph.nodes:
    x = x_s+sep_dist*n[0]
    y = y_s-sep_dist*n[1]
    
    latlon = utm.to_latlon(x,y, z, zl)
    
    lons.append(latlon[1])
    lats.append(latlon[0])
    
    Xs.append(x)
    Ys.append(y)
    
    g.add_node(node_num)
    
    ind[n] = node_num
    
    New_Long_Lat[node_num] = [latlon[1], latlon[0]]
    
    node_num += 1

In [12]:
for n in grid_graph.nodes:
    for nei in grid_graph.neighbors(n):
        g.add_edge(ind[n], ind[nei])

In [13]:
for i in grid_graph.nodes:
    for j in range(Num_Vertiports):
        dist = ((Xs[ind[i]]-Xs[j])**2 + (Ys[ind[i]]-Ys[j])**2)**0.5
        
        if dist < 1.2*sep_dist:
            g.add_edge(ind[i],j)

In [14]:
SRC_list = []
DST_list = []

for e in g.edges:
    SRC_list.append([lons[int(e[0])], lats[int(e[0])]])
    DST_list.append([lons[int(e[1])], lats[int(e[1])]])
    
dictv = {'start': SRC_list, 'end': DST_list} 

edge_df = pd.DataFrame(dictv)
    

In [15]:
full_df = {}


Node_Lon_Lat = []
Node_Color = []
Size = []
Vertiport = []
Tooltip = []


for i in range(len(lons)):

    if (i < Num_Vertiports):
        Node_Lon_Lat.append([lons[i], lats[i]])
        Node_Color.append([0, 255, 0, 255])
        Size.append(1000)
        Tooltip.append(f"<b>Type:</b> Vertiport: {i}")

    if (i >= Num_Vertiports):

        Node_Lon_Lat.append([lons[i], lats[i]])
        Node_Color.append([0, 255, 0, 50])
        Size.append(1.5)
        Tooltip.append(f"<b>Type:</b> Node: {i}")


dictv = {'coordinates': Node_Lon_Lat, 'color': Node_Color, 'size': Size, 'tooltip': Tooltip} 

df = pd.DataFrame(dictv)


In [16]:

sunlight = {
    "@@type": "_SunLight",
    "timestamp": 1564696800000,  # Date.UTC(2019, 7, 1, 22),
    "color": [255, 255, 255],
    "intensity": 1.0,
    "_shadow": True,
}

ambient_light = {"@@type": "AmbientLight", "color": [255, 255, 255], "intensity": 1.0}

lighting_effect = {
    "@@type": "LightingEffect",
    "shadowColor": [0, 0, 0, 0.5],
    "ambientLight": ambient_light,
    "directionalLights": [sunlight],
}

view_state = pdk.ViewState(
    **{"latitude": JFK_LAT_LON[0], "longitude": JFK_LAT_LON[1], "zoom": 10, "maxZoom": 14, "pitch": 90, "bearing": 0}
)




layer = pdk.Layer(
    "ScatterplotLayer",
    df,
    pickable=True,
    stroked=True,
    filled=True,
    radius_scale=0.1,
    radius_min_pixels=1,
    radius_max_pixels=100,
    line_width_min_pixels=1,
    get_position="coordinates",
    get_radius="size",
    get_fill_color="color",
    get_line_color=[0, 0, 0],
)

line_layer = pdk.Layer(
    "LineLayer",
    edge_df,
    get_source_position="start",
    get_target_position="end",
    get_color=[105,105,105],
    get_width=0.5,
    highlight_color=[255, 255, 0],
    picking_radius=10,
    auto_highlight=True,
)

tooltip = {"html": "{tooltip}"}

keys = {"mapbox": "pk.eyJ1IjoibWNkb25zdCIsImEiOiJjbDRyNGphOWwweHA1M2lxemI1ZXhmM3Y1In0.Km2Tt6dKpTqwfNmTMEKrQA"}


layers = [line_layer, layer]

r = pdk.Deck(
    layers,
    initial_view_state=view_state,
    effects=[lighting_effect],
    map_style="dark_no_labels",
    api_keys=keys,
    tooltip=tooltip,
)




# # display and save map (to_html(), show())
r.show()

DeckGLWidget(carto_key=None, custom_libraries=[], google_maps_key=None, json_input='{"effects": [{"@@type": "L…

In [17]:
Adj = np.zeros((len(lons),len(lons)))

In [18]:
for e in g.edges:

    dist = ((Xs[e[0]]-Xs[e[1]])**2 + (Ys[e[0]]-Ys[e[1]])**2)**0.5
    Adj[e[0],e[1]] = dist
    Adj[e[1],e[0]] = dist

In [19]:

with open('LongLat.json', 'w') as f:
    json.dump(New_Long_Lat, f)
nx.set_node_attributes(g, New_Long_Lat, name="pos")

In [20]:
nx.write_gml(g, "Grid.gml")

In [21]:
Edges = {}
k = 1
for e in g.edges:
    Edges[k] = [e[0]+1, e[1]+1, Adj[e[0],e[1]]]
    k += 1
    
Edges["Nodes"] = g.number_of_nodes()
Edges["Edges"] = g.number_of_edges()


In [22]:
with open('Edges.json', 'w') as f:
    json.dump(Edges, f)

In [23]:
Edges

{1: [1, 367, 1258.9365180230172],
 2: [1, 391, 1563.5970484305806],
 3: [1, 392, 363.7236785507152],
 4: [1, 393, 1731.9729557612059],
 5: [2, 261, 1104.2082117261923],
 6: [2, 262, 1491.7706408648394],
 7: [2, 286, 792.2155964508612],
 8: [2, 287, 1278.1664294459981],
 9: [3, 268, 1023.4109286109264],
 10: [3, 269, 759.0259641467304],
 11: [3, 293, 1555.9563008279197],
 12: [3, 294, 1396.334664589653],
 13: [4, 291, 675.1507072404637],
 14: [4, 292, 1119.8926248449695],
 15: [4, 316, 1347.608943094875],
 16: [4, 317, 1616.9047211274212],
 17: [5, 333, 1187.9192053259667],
 18: [5, 334, 586.723192337969],
 19: [5, 358, 1698.4219086296932],
 20: [5, 359, 1348.2318219647211],
 21: [6, 270, 1898.7507855625577],
 22: [6, 271, 1256.52215543337],
 23: [6, 295, 1479.983101566816],
 24: [6, 296, 404.89920022621106],
 25: [6, 297, 1821.4041189248078],
 26: [7, 220, 1618.228771346522],
 27: [7, 221, 1139.9354097828473],
 28: [7, 245, 1329.1530631376693],
 29: [7, 246, 668.9067553189778],
 30: [8